<a href="https://colab.research.google.com/github/nsajwan/line_item_extraction/blob/master/Line_Item_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bank statements Line Item Extraction
Based on Cascade TableNet - https://github.com/DevashishPrasad/CascadeTabNet 

## 1. Install all the prerequisite
This will take a while, Remember to **Restart Runtime** after this step is done 

In [19]:
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q mmcv terminaltables
!git clone --branch v1.2.0 'https://github.com/open-mmlab/mmdetection.git'
%cd "mmdetection"
!pip install -r "/content/mmdetection/requirements/optional.txt"
!python setup.py install
!python setup.py develop
!pip install -r {"requirements.txt"}
!pip install pillow==6.2.1 
!pip install mmcv==0.4.3
%cd "/content"
!apt-get install poppler-utils
!pip install pdf2image
!pip install pdfminer
!pip install Pillow

Looking in links: https://download.pytorch.org/whl/torch_stable.html
fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
running install
running bdist_egg
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying mmdet/version.py -> build/lib.linux-x86_64-3.6/mmdet
running build_ext
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mmdet
creating build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib.linux-x86_64-3.6/mmdet/utils/registry.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying build/lib.linux-x86_64-3.6/mmdet/utils/logger.py -> build/bdist.linux-x86_64/egg/mmdet/utils
copying buil

## 2. Clone the Repository

In [1]:
!rm -rf line_item_extraction
!git clone https://github.com/strangest-quark/line_item_extraction.git

Cloning into 'line_item_extraction'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 84 (delta 34), reused 45 (delta 12), pack-reused 0
Unpacking objects: 100% (84/84), done.


## 3. Pdf to Images and XML


In [2]:
!rm -rf /content/line_item_extraction/results
import os
from pdf2image import convert_from_path, convert_from_bytes
from PIL import Image
import subprocess

pdf_directory = '/content/line_item_extraction/sample_docs'
res_directory = '/content/line_item_extraction/results'
cmd = 'pdftohtml -i -c -noframes -xml '

results = []

for filename in os.listdir(pdf_directory):
  try:
    if filename.endswith(".pdf"):
      print(filename)

      os.makedirs(res_directory+'/'+filename[:-4])
      full_cmd = cmd+pdf_directory+'/'+filename+' '+res_directory+'/'+filename[:-4]+'/'+filename[:-4]+'.xml'
      print(full_cmd)
      print(" ")
      !{full_cmd}
      images = convert_from_path(pdf_directory+'/'+filename)
      i = 0
      print(i)
      os.makedirs(res_directory+'/'+filename[:-4]+'/img')
      for image in images:
        image.save(res_directory+'/'+filename[:-4]+'/img/'+str(i)+'.png', format='PNG')
        i=i+1
  except:
    print("Error")

Redacted_BB&T_2_Sample2.pdf
pdftohtml -i -c -noframes -xml /content/line_item_extraction/sample_docs/Redacted_BB&T_2_Sample2.pdf /content/line_item_extraction/results/Redacted_BB&T_2_Sample2/Redacted_BB&T_2_Sample2.xml
 
/bin/bash: T_2_Sample2.xml: command not found
/bin/bash: T_2_Sample2.pdf: command not found
/bin/bash: T_2_Sample2/Redacted_BB: No such file or directory
I/O Error: Couldn't open file '/content/line_item_extraction/sample_docs/Redacted_BB': No such file or directory.
0
Redacted_CIBC_2_Sample2.pdf
pdftohtml -i -c -noframes -xml /content/line_item_extraction/sample_docs/Redacted_CIBC_2_Sample2.pdf /content/line_item_extraction/results/Redacted_CIBC_2_Sample2/Redacted_CIBC_2_Sample2.xml
 
Page-1
Page-2
0
Redacted_RoyalBankCanada.pdf
pdftohtml -i -c -noframes -xml /content/line_item_extraction/sample_docs/Redacted_RoyalBankCanada.pdf /content/line_item_extraction/results/Redacted_RoyalBankCanada/Redacted_RoyalBankCanada.xml
 
Page-1
0
RedactedPayPal_sample3.pdf
pdftohtml -

## 3. Download the Pretrained Model


In [3]:
!gdown "https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT"

Downloading...
From: https://drive.google.com/u/0/uc?id=1-QieHkR1Q7CXuBu4fp3rYrvDG9j26eFT
To: /content/epoch_36.pth
664MB [00:02, 236MB/s]


## 4. Run the Predictions

In [4]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
# Load model
config_file = '/content/line_item_extraction/config/cascade_mask_rcnn_hrnetv2p_w32_20e.py'
checkpoint_file = '/content/epoch_36.pth'
# build the model from a config file and a checkpoint file
"""Initialize a detector from config file.
Args:
    Config file path
    Checkpoint path. If left as None, the model  
        will not load any weights.  
Returns:
    nn.Module: The constructed detector."""
model = init_detector(config_file, checkpoint_file, device='cuda:0')

directory = '/content/line_item_extraction/sample_docs/imgs'

results = []

for filename in os.listdir(directory):
    if filename.endswith(".png"):
      # Run Inference
      """ 
      Inference image(s) with the detector.
   Args:
    model (nn.Module): The loaded detector.  
    imgs (str/ndarray or list[str/ndarray]): Either image files or loaded  
        images.  
  Returns:
    If imgs is a str, a generator will be returned, otherwise return the  
    detection results directly.
      """
      results.append(inference_detector(model, directory+'/'+filename))
      # Visualization results
      """
      Visualize the detection results on the image.
Args:
    img (str or np.ndarray): Image filename or loaded image.  
    result (tuple[list] or list): The detection result, can be either  
        (bbox, segm) or just bbox.  
    class_names (list[str] or tuple[str]): A list of class names.  
    score_thr (float): The threshold to visualize the bboxes and masks.  
    fig_size (tuple): Figure size of the pyplot figure.  
    out_file (str, optional): If specified, the visualization result will  
        be written to the out file instead of shown in a window.  
      """
      show_result_pyplot(directory+'/'+filename, results[-1],('Bordered', 'cell', 'Borderless'), score_thr=0.85)

ImportError: ignored

After this, use TSR postprocessing for best result

In [ ]:
%cd "/content"
from line_item_extraction.border_main import border
import lxml.etree as etree
import cv2
from google.colab.patches import cv2_imshow
from mmdet.apis import inference_detector, show_result, init_detector
import glob
from line_item_extraction.cell_text import borderless

result = results[0]

image_path = '/content/line_item_extraction/results/RedactedPayPal_sample3/img/'

config_fname = "/content/line_item_extraction/config/cascade_mask_rcnn_hrnetv2p_w32_20e.py" 
checkpoint_path = "/content/"
epoch = 'epoch_36.pth'

model = init_detector(config_fname, checkpoint_path+epoch)

imgs=[]

# List of images in the image_path
for filename in os.listdir(image_path):
    if filename.endswith(".png"):
      imgs.append(image_path+filename)
print(imgs)
for i in imgs:
  for i in imgs:
    try:
      result = inference_detector(model, i)
      print("result",result)
      res_border = []
      res_bless = []
      res_cell = []
      root = etree.Element("document")
      print("root=="+root);
      ## for border
      for r in result[0][0]:
        if r[4] > .85:
            res_border.append(r[:4].astype(int))
      ## for cells
      for r in result[0][1]:
          if r[4] > .85:
              r[4] = r[4] * 100
              res_cell.append(r.astype(int))
      ## for borderless
      for r in result[0][2]:
          if r[4] > .85:
              res_bless.append(r[:4].astype(int))

      ## if border tables detected
      if len(res_border) != 0:
          ## call border script for each table in image
          for res in res_border:
              try:
                  root.append(border(res, cv2.imread(i)))
              except:
                  pass
      if len(res_bless) != 0:
          if len(res_cell) != 0:
              for no, res in enumerate(res_bless):
                  root.append(borderless(res, cv2.imread(i), res_cell))
      myfile = open(xmlPath + i.split('/')[-1][:-3] + 'xml', "w")
      myfile.write('<?xml version="1.0" encoding="UTF-8"?>\n')
      myfile.write(etree.tostring(root, pretty_print=True, encoding="unicode"))
      myfile.close()
    except:
      print("Error")
      

In [ ]:
!pip install pdfrw
from pdfrw import PdfReader
pdf = PdfReader('/content/line_item_extraction/sample_docs/RedactedPayPal_sample3.pdf')
from PIL import Image
im = Image.open('/content/line_item_extraction/results/RedactedPayPal_sample3/img/0.png')

def TranslatePoints(src, pdf, im):

        sx0, sy0, sx1, sy1 = src
        print(len(pdf.pages))
        ssx, ssy = (int(pdf.pages[0].MediaBox[2]),int(pdf.pages[0].MediaBox[3]))
        print(ssx, ssy)
        dsx, dsy = im.size
        sy01 = ssy-sy1
        sy11 = ssy-sy0
        x0 = sx0/int(pdf.pages[0].MediaBox[2])*im.size[0]
        x1 = sx1/int(pdf.pages[0].MediaBox[2])*im.size[0]
        y0 = sy01/int(pdf.pages[0].MediaBox[3])*im.size[1]
        y1 = sy11/int(pdf.pages[0].MediaBox[3])*im.size[1]
        return (x0, y0, x1, y1)


TranslatePoints((46, 876, 105, 887), pdf, im)